In [2]:
%config Completer.use_jedi = False
from scipy import integrate
import numpy as np
import math

## 1.1 Integration with Scipy

The scipy.integrate sub-package provides several integration techniques including an ordinary differential equation integrator. 

    Methods for Integrating Functions given function object.

       quad          -- General purpose integration.
       dblquad       -- General purpose double integration.
       tplquad       -- General purpose triple integration.
       fixed_quad    -- Integrate func(x) using Gaussian quadrature of order n.
       quadrature    -- Integrate with given tolerance using Gaussian quadrature.
       romberg       -- Integrate func using Romberg integration.

    Methods for Integrating Functions given fixed samples.

       trapz         -- Use trapezoidal rule to compute integral from samples.
       cumtrapz      -- Use trapezoidal rule to cumulatively compute integral.
       simps         -- Use Simpson's rule to compute integral from samples.
       romb          -- Use Romberg Integration to compute integral from
                        (2**k + 1) evenly-spaced samples.

       See the special module's orthogonal polynomials (special) for Gaussian
          quadrature roots and weights for other weighting factors and regions.

    Interface to numerical integrators of ODE systems.

       odeint        -- General integration of ordinary differential equations.
       ode           -- Integrate ODE using VODE and ZVODE routines.

### 1.1.1 General integration (quad)

The function quad (quadrature integration) is provided to integrate a function of one variable between two points. The points can be ±∞ (± np.inf) to indicate infinite limits. 

---
**Example 1:** Suppose you wish to integrate a standard normal distribution:

$$\Phi \left( Z \right) = \int_{ - \infty }^Z {\frac{1}{\sqrt {2\pi }} e^{ - \frac{x^2}{2}}} dx$$

In [3]:
def rho(x):
    return math.exp(-0.5 * x**2.) / math.sqrt(2*math.pi)

result = integrate.quad(rho,-np.inf, 0)
result

(0.4999999999999999, 5.089095660452871e-09)

The first argument is the result of the integration and the second argument in the result is the upper bound on the error.

---

**Example 2:** Handling multiple integration with quad function

$$E_{n}\left(x\right)=\int_{1}^{\infty}\frac{e^{-xt}}{t^{n}}\, dt$$

$$I_{n}=\int_{0}^{\infty}\int_{1}^{\infty}\frac{e^{-xt}}{t^{n}}\, dt\, dx=\frac{1}{n}$$

In [4]:
def integrand(t, n, x):
    return math.exp(-x*t) / t**n

def expint(n, x): #E(n,x)
    r_enx = integrate.quad(integrand, 1, np.inf, args=(n, x))
    return r_enx[0]

result1 = integrate.quad(lambda x: expint(3, x), 0, np.inf)
print("result1 = ", result1)

# Alternative method without using lamda expression
def n_expint(x):
    return expint(3, x)

result2 = integrate.quad(n_expint, 0, np.inf)
print("result2 = ", result2)

result1 =  (0.33333333325010883, 2.8604069921197956e-09)
result2 =  (0.33333333325010883, 2.8604069921197956e-09)


---

**Exercise 1:** Evaluate the integral

$$\int_0^\infty  {\frac{1}{1 + {x^2}}} dx$$


---

**Exercise 2:** The total power radiated per unit area is given by:

$$I = \sigma T^4$$

or more precisely,

$$I ={2\pi(k T)^4 \over c^2h^3}\int_0^\infty dx {x^3 \over e^x-1}$$

Show that $\sigma = 5.67 \times 10^{-8} W m^{-2} K^{-4}$.

where,

$k$ is the Boltzmann constant

$c$ is the Speed of light in vacuum

$h$ is the Planck constant

(These constants can be imported from scipy)

In [5]:
from scipy import constants
k = constants.k     # Boltzmann constant
c = constants.c     # Speed of light in vacuum
h = constants.h     # the Planck constant

---
### 1.1.2 General multiple integration (dblquad, tplquad, nquad)

The mechanics for double and triple integration have been wrapped up into the functions dblquad and tplquad. These functions take the function to integrate and four, or six arguments, respectively. The limits of all inner integrals need to be defined as functions.

Use the example intergral in the previous section for $I_n$:

$$I_{n}=\int_{0}^{\infty}\int_{1}^{\infty}\frac{e^{-xt}}{t^{n}}\, dt\, dx=\frac{1}{n}$$

In [6]:
def I(n):
    return integrate.dblquad(lambda t, x: math.exp(-x*t)/t**n, 0, np.inf, lambda x: 1, lambda x: np.inf)

I(3)

(0.33333333325010883, 1.3888461883425516e-08)

In [7]:
def I(n):
    def func(t, x):
        return np.exp(-x*t)/t**n

    def bound_a(x):
        return 1

    def bound_b(x):
        return np.inf
    result = integrate.dblquad(func, 0, np.inf, bound_a, bound_b)
    return result


I(3)

(0.33333333325010883, 1.3888461883425516e-08)

---
**Example 3:** Non-constant limits consider the integral

$$I=\int_{y=0}^{1/2}\int_{x=0}^{1-2y} x y \, dx\, dy=\frac{1}{96}$$

In [8]:
area = integrate.dblquad(lambda x, y: x*y, 0, 0.5, lambda x: 0, lambda y: 1-2*y)

print(1./96)
area[0]

0.010416666666666666


0.010416666666666668

---
For n-fold integration, scipy provides the function nquad. The integration bounds are an iterable object: either a list of constant bounds, or a list of functions for the non-constant integration bounds. The order of integration (and therefore the bounds) is from the innermost integral to the outermost one.

---
**Example 4**: n-fold integration of integral from above

$$I_{n}=\int_{0}^{\infty}\int_{1}^{\infty}\frac{e^{-xt}}{t^{n}}\, dt\, dx=\frac{1}{n}$$

In [9]:
N = 3

def f(t, x):
    return np.exp(-x*t) / t**N

integrate.nquad(f, [[1, np.inf],[0, np.inf]])

(0.33333333325010883, 1.3888461883425516e-08)

---
**Example 5:** n-fold integration of non-constant limits consider the integral

In [10]:
def f(x, y):
    return x*y

def bounds_y():
    return [0, 0.5]

def bounds_x(y):
    return [0, 1-2*y]

integrate.nquad(f, [bounds_x, bounds_y])[0]

0.010416666666666668

Same result as above!

---
**Exercise 3:**

$$I=\int_{y=0}^{1}\int_{x=0}^{2} {e^{-xy}}\, dx\, dy$$

In [11]:
def f(x, y):
    return np.exp(-x*y)


def bounds_y():
    return [0, 1]


def bounds_x(x):
    return [0, 2]


integrate.nquad(f, [bounds_x, bounds_y])[0]

1.3192633561695395

---
**Exercise 4:**

$$I=\int_{y=0}^{1}\int_{x=0}^{\sqrt{1-y^2}} {e^{-xy}}\, dx\, dy$$

In [12]:
def f(x, y):
    return np.exp(-x*y)


def bounds_y():
    return [0, 1]


def bounds_x(y):
    return [0, np.sqrt(1-y**2)]


integrate.nquad(f, [bounds_x, bounds_y])[0]

0.6751670568500855

What does non-constant limits do to the domain of integration?